# Testing File
---
_This notebook is for testing new functionality before introducing it into the program._

### Protocol
---
_Pokémon Showdown is accessible via websockets, so I can interact with the website from this file (sending messages, logging in, and so on)._
_Found here: https://github.com/smogon/pokemon-showdown/blob/master/PROTOCOL.md_

**Variables I'll need**:
* Websocket UI
* Username and Password
* Format
* Team to use (if applicable)

In [1]:
import asyncio
import websockets

uri = "wss://sim3.psim.us/showdown/websocket"
login-uri = "https://play.pokemonshowdown.com/action.php"
testusername = "thisisatest12345"
testpassword = "password" # very secure :)

